In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import random
import pickle
from randomforest_regression_std import *
from sklearn.externals.joblib import Parallel, delayed

In [ ]:
def read_features(text_file):
    f = open(text_file, 'r')
    x = f.readlines()
    f.close()
    features=[]
    for i in x:
        features.append(i[:-1])
    return features

### Read different features

In [ ]:
meta = ['Reference DOI','Composition ID']

coercivity_feature_file='kept_coercivity.txt'
magnetostriction_feature_file='kept_magnetostriction.txt'
curietemp_feature_file='kept_curie_temp.txt'

coercivity = pd.read_csv('Coercivity7-26.csv').drop(columns=meta)
magnetostriction = pd.read_csv('Magnetostriction7-26.csv').drop(columns=meta)
curietemp = pd.read_csv('CurieTemperature7-26.csv').drop(columns=meta)

coercivity_feature=read_features(coercivity_feature_file)
magnetostriction_feature=read_features(magnetostriction_feature_file)
curietemp_feature=read_features(curietemp_feature_file)

coercivity_target=np.log(coercivity.iloc[:, -1])
magnetostriction_target=magnetostriction.iloc[:, -1]
curietemp_target=curietemp.iloc[:, -1]

coercivity=coercivity[coercivity_feature]
coercivity=coercivity.drop(columns=["Relative to Early SiCAl", "Relative to Late SiCAl","Relative to Early BP", "Relative to Late BP", "Early Weighted Volume", "Early Weighted Mass",\
                                    "Primary Crystallization Peak (K)", "Delta T1"  ])
magnetostriction=magnetostriction[magnetostriction_feature]
magnetostriction=magnetostriction.drop(columns=["Relative to Fe BP","Relative to Early BP","Early Weighted Volume","Early Weighted Mass" ])
curietemp=curietemp[curietemp_feature]
curietemp=curietemp.drop(columns=["Primary Crystallization Onset (K)", "Early Weighted Mass"])



magneticsaturation_feature_file='kept_magnetic_saturation.txt'
magneticsaturation = pd.read_csv('MagneticSaturation7-26.csv').drop(columns=meta)
magneticsaturation_feature=read_features(magneticsaturation_feature_file)
magneticsaturation_target=magneticsaturation.iloc[:, -1]
magneticsaturation=magneticsaturation[magneticsaturation_feature]
magneticsaturation=magneticsaturation.drop(columns=["Total BP", "Delta T2"])
magneticsaturation=magneticsaturation.fillna(magneticsaturation.mean())

### Find the feature set that include both coercivity and magnetostriction features

In [ ]:
def merge_features(a,b):
    aa=np.array(a)
    bb=np.array(b)
    new_array = np.unique(np.concatenate((aa,bb),0))
    return new_array

mergedf=merge_features(list(coercivity.columns.values),list(magnetostriction.columns.values))

mergedf=merge_features(mergedf,list(magneticsaturation.columns.values))

### Features now are union set of two properties, set 0 for features with no data

In [ ]:
def expand_data(frame, features):
    newframe=pd.DataFrame()
    for i in features:
        if i not in list(frame):
            newframe[i]=0
        else:
            newframe[i]=frame[i]
    return newframe

coercivity=expand_data(coercivity, mergedf)
magnetostriction=expand_data(magnetostriction,mergedf)
curietemp=expand_data(curietemp,mergedf)
magneticsaturation=expand_data(magneticsaturation, mergedf)

### Random forest model

In [ ]:
def model_rf(feature,target):
    rf = RandomForestRegressor(n_estimators=1000, min_samples_leaf=1,min_variance=0.01)
    feature.fillna(0, inplace=True)
    X=feature.as_matrix()
    Y=target.as_matrix()
    rf.fit(X, Y)
    return rf, X

#### Model for Coercivity

In [ ]:
coercivitymodel, coercivitydata= model_rf(coercivity, coercivity_target)
preds=coercivitymodel.predict(coercivitydata)
axismax=max(preds.max(),coercivity_target.max())
axismin=min(preds.min(),coercivity_target.min())
plt.plot(preds, coercivity_target,'o')
plt.plot([axismin, axismax], [axismin, axismax], 'k--')
plt.ylabel('Reported ')
plt.xlabel('Predicted ')
plt.show()

In [ ]:
# Feature importances
importances = coercivitymodel.feature_importances_
indices = np.argsort(importances)[::-1]

plt.figure(figsize=[10,8])
plt.title("Feature importances")
plt.bar(range(coercivitydata.shape[1]), importances[indices],
       color="r", align="center")
plt.xticks(range(coercivitydata.shape[1]), indices,fontsize=18)
plt.xlim([-1, coercivitydata.shape[1]])
plt.show()

#### Other properties

In [ ]:
magnetomodel,magnetodata=model_rf(magnetostriction, magnetostriction_target)
curietempmodel,curietempdata=model_rf(curietemp, curietemp_target)
magneticsaturationmodel, magneticsaturationdata= model_rf(magneticsaturation, magneticsaturation_target)

In [ ]:
# plot for curie temperature
preds=curietempmodel.predict(curietempdata)
axismax=max(preds.max(),curietemp_target.max())
axismin=min(preds.min(),curietemp_target.min())
plt.plot(preds, curietemp_target,'o')
plt.plot([axismin, axismax], [axismin, axismax], 'k--')
plt.ylabel('Reported ')
plt.xlabel('Predicted ')
plt.show()

### Composition constraints, identify elements in features and make them add up to less than 100

In [ ]:
def identify_element(mergedf):
    A=[]
    for i in mergedf:
        if len(i)<3:
            A.append(1)
        else:
            A.append(0)
    return np.array(A)


A=identify_element(mergedf).reshape(1,-1)
lb=-1
ub=100.01

lowerboundsc=np.min(coercivitydata,axis=0)
lowerboundsmagneto=np.min(magnetodata,axis=0)
lowerboundssat=np.min(magneticsaturationdata,axis=0)
lowerbounds=np.zeros(len(lowerboundsc))
for i in range(len(lowerboundsc)):
    if lowerboundsc[i]==0 or lowerboundsmagneto[i]==0:
        lowerbounds[i]=max(lowerboundsc[i],lowerboundsmagneto[i])
    else:
        lowerbounds[i]=min(lowerboundsc[i],lowerboundsmagneto[i])
    
    if lowerboundssat[i]==0 or lowerbounds[i]==0:
        lowerbounds[i]=max(lowerbounds[i],lowerboundssat[i])
    else:
        lowerbounds[i]=min(lowerbounds[i],lowerboundssat[i])
    

upperboundsc=np.max(coercivitydata,axis=0)
upperboundsmagneto=np.max(magnetodata,axis=0)
upperboundssat=np.max(magneticsaturationdata,axis=0)

upperbounds=np.zeros(len(upperboundsc))
for i in range(len(upperboundsc)):
    if upperboundsc[i]==0 or upperboundsmagneto[i]==0:
        upperbounds[i]=max(upperboundsc[i],upperboundsmagneto[i])
    else:
        upperbounds[i]=max(upperboundsc[i],upperboundsmagneto[i])
    
    if upperboundssat[i]==0:
        upperbounds[i]=max(upperbounds[i],upperboundssat[i])
    else:
        upperbounds[i]=max(upperbounds[i],upperboundssat[i])

# Bounds on all features
for i, num in enumerate(upperbounds):
    if num==0:
        upperbounds[i]=100

## First optimization strategy
#### Constrain ln(coercivity) < -1.5; Constrain magnetostriction < 3; maximizing magnetic saturation

In [ ]:
def optimingfunction(X,coercivitymodel,magnetomodel,magneticsaturationmodel):
    magnetopred=abs(magnetomodel.predict(X.reshape(1,-1), return_std=False))
    compsum=np.dot(A,X.reshape(-1,1))
    coercivitypred=coercivitymodel.predict(X.reshape(1,-1), return_std=False)
    magneticsatpred=magneticsaturationmodel.predict(X.reshape(1,-1), return_std=False)
    #print(magneticsatpred)
    if compsum>100:
        return (compsum-100)**2+100
    if  magnetopred > 3 :
        return abs(magnetopred-3)**2+50-magneticsatpred
    if  coercivitypred >-1.5:
        return abs(coercivitypred+1.5)**2+50-magneticsatpred
    
    return -magneticsatpred
    

In [ ]:
'''Run optimizations based on different combination of element composition constraints'''
diffbet=[]

results=[]
datass=[]
from scipy.optimize import differential_evolution

bounds=[]
for i in range(lowerbounds.shape[0]):
    bb=(lowerbounds[i],upperbounds[i])
    bounds.append(bb)

X0paths=[]  
X0path=[]
ypaths=[]
ypath=[]
def callbackF(Xi,convergence=0.05):
    X0path.append(Xi)
    ypath.append(optimingfunction(Xi,coercivitymodel,magnetomodel,magneticsaturationmodel))
    return 
strategies=["best1bin"]

indas=[[2],[2],[2],[2],[2],[2]]
indbs=[[7,8,12],[7,8,12],[None],[7,8,12],[7,8,12],[None]]
indcs=[[None],[7,8,12],[None],[None],[7,8,12],[None]]
indds=[[6,13],[6,13],[6,13],[6],[6],[6]]

for consi in range(6):
    inda=indas[consi]
    indb=indbs[consi]
    indc=indcs[consi]
    indd=indds[consi]

    for jj in inda :
        for ii in indb:
            for iii in indc:
                if ii==iii:
                    continue
                bb=bounds[:]
                bb[jj]=(0,0)
                if ii!=None:
                    bb[ii]=(0,0)
                if iii!=None:
                    bb[iii]=(0,0)
                for sind in indd:
                    bb[sind]=(0,0)

                for chosestrategy in strategies:
                    X0path=[]
                    ypath=[]
                    result=differential_evolution(optimingfunction,bb,args=(coercivitymodel,magnetomodel,magneticsaturationmodel), \
                                                  strategy=chosestrategy,popsize=30,mutation=(0.7,1.5),recombination=0.5, callback=callbackF, disp=1)
                    X0paths.append(X0path[:])
                    ypaths.append(ypath[:])
                    results.append(result)
                    datass.append(result['x'])

In [ ]:
'''get prediction value after optimization '''
predslist=[]
magstriction=[]
curietemppreds=[]
magneticsaturationpreds=[]
for data in datass:
    predss=coercivitymodel.predict(data.reshape(1,-1))
    magpred=magnetomodel.predict(data.reshape(1,-1))
    
    predslist.append(predss[0])
    magstriction.append(magpred[0])
    curietemppreds.append( curietempmodel.predict(data.reshape(1,-1))[0])
    magneticsaturationpreds.append( magneticsaturationmodel.predict(data.reshape(1,-1))[0])
magstriction=np.array(magstriction)
curietemppreds=np.array(curietemppreds)
magneticsaturationpreds=np.array(magneticsaturationpreds)
'''Sorting the optimized prediction ascending '''    
indexpred=np.argsort(predslist)
datass=np.array(datass)
datass2=datass[indexpred]
columnsname=coercivity.columns.values
optimizedframe=pd.DataFrame(data=datass2,columns=columnsname)
optimizedframe['coercivity']=sorted(predslist)
optimizedframe['magnetostriction']=magstriction[indexpred]
optimizedframe['curietemp']=curietemppreds[indexpred]
optimizedframe['magnetic-saturation']=magneticsaturationpreds[indexpred]
writer = pd.ExcelWriter('output_1.xlsx')
optimizedframe.to_excel(writer,'Sheet1')
writer.save()


## Second optimization strategy
####  Constrain magnetostriction < 3 ;    Minimizing (-magnetic saturation+coercivity)

In [ ]:
def optimingfunction(X,coercivitymodel,magnetomodel,magneticsaturationmodel):
    magnetopred=abs(magnetomodel.predict(X.reshape(1,-1), return_std=False))
    compsum=np.dot(A,X.reshape(-1,1))
    coercivitypred=coercivitymodel.predict(X.reshape(1,-1), return_std=False)
    magneticsatpred=magneticsaturationmodel.predict(X.reshape(1,-1), return_std=False)
    #print(magneticsatpred)
    if compsum>100:
        return (compsum-100)**2+100
    if  magnetopred > 3 :
        return abs(magnetopred-3)**2+50-magneticsatpred
    
    return -magneticsatpred+coercivitypred

In [ ]:
'''Run optimizations based on different combination of element composition constraints'''


diffbet=[]

results=[]
datass=[]
from scipy.optimize import differential_evolution

bounds=[]
for i in range(lowerbounds.shape[0]):
    bb=(lowerbounds[i],upperbounds[i])
    bounds.append(bb)

X0paths=[]  
X0path=[]
ypaths=[]
ypath=[]
def callbackF(Xi,convergence=0.05):
    X0path.append(Xi)
    ypath.append(optimingfunction(Xi,coercivitymodel,magnetomodel,magneticsaturationmodel))
    return 
strategies=["best1bin"]

indas=[[2],[2],[2],[2],[2],[2]]
indbs=[[7,8,12],[7,8,12],[None],[7,8,12],[7,8,12],[None]]
indcs=[[None],[7,8,12],[None],[None],[7,8,12],[None]]
indds=[[6,13],[6,13],[6,13],[6],[6],[6]]

for consi in range(6):
    inda=indas[consi]
    indb=indbs[consi]
    indc=indcs[consi]
    indd=indds[consi]

    for jj in inda :
        for ii in indb:
            for iii in indc:
                if ii==iii:
                    continue
                bb=bounds[:]
                bb[jj]=(0,0)
                if ii!=None:
                    bb[ii]=(0,0)
                if iii!=None:
                    bb[iii]=(0,0)
                for sind in indd:
                    bb[sind]=(0,0)

                for chosestrategy in strategies:
                    X0path=[]
                    ypath=[]
                    result=differential_evolution(optimingfunction,bb,args=(coercivitymodel,magnetomodel,magneticsaturationmodel), \
                                                  strategy=chosestrategy,popsize=30,mutation=(0.7,1.5),recombination=0.5, callback=callbackF, disp=1)
                    X0paths.append(X0path[:])
                    ypaths.append(ypath[:])
                    results.append(result)
                    datass.append(result['x'])

In [ ]:
'''get prediction value after optimization '''
predslist=[]
magstriction=[]
curietemppreds=[]
magneticsaturationpreds=[]
for data in datass:
    predss=coercivitymodel.predict(data.reshape(1,-1))
    magpred=magnetomodel.predict(data.reshape(1,-1))
    
    predslist.append(predss[0])
    magstriction.append(magpred[0])
    curietemppreds.append( curietempmodel.predict(data.reshape(1,-1))[0])
    magneticsaturationpreds.append( magneticsaturationmodel.predict(data.reshape(1,-1))[0])
magstriction=np.array(magstriction)
curietemppreds=np.array(curietemppreds)
magneticsaturationpreds=np.array(magneticsaturationpreds)
'''Sorting the optimized prediction ascending '''    
indexpred=np.argsort(predslist)
datass=np.array(datass)
datass2=datass[indexpred]
columnsname=coercivity.columns.values
optimizedframe=pd.DataFrame(data=datass2,columns=columnsname)
optimizedframe['coercivity']=sorted(predslist)
optimizedframe['magnetostriction']=magstriction[indexpred]
optimizedframe['curietemp']=curietemppreds[indexpred]
optimizedframe['magnetic-saturation']=magneticsaturationpreds[indexpred]
writer = pd.ExcelWriter('output_2.xlsx')
optimizedframe.to_excel(writer,'Sheet1')
writer.save()


### Third optimization strategy
#### Constrain ln(coercivity) < -0.5; Constrain magnetostriction < 3; maximizing magnetic saturation.

In [ ]:
def optimingfunction(X,coercivitymodel,magnetomodel,magneticsaturationmodel):
    magnetopred=abs(magnetomodel.predict(X.reshape(1,-1), return_std=False))
    compsum=np.dot(A,X.reshape(-1,1))
    coercivitypred=coercivitymodel.predict(X.reshape(1,-1), return_std=False)
    magneticsatpred=magneticsaturationmodel.predict(X.reshape(1,-1), return_std=False)
    
    if compsum>100:
        return (compsum-100)**2+100
    if  magnetopred > 3 :
        return abs(magnetopred-3)**2+50-magneticsatpred
    if  coercivitypred >-0.5:
        return abs(coercivitypred+0.5)**2+5
    
    return -magneticsatpred

In [ ]:
'''Run optimizations based on different combination of element composition constraints'''


diffbet=[]

results=[]
datass=[]
from scipy.optimize import differential_evolution

bounds=[]
for i in range(lowerbounds.shape[0]):
    bb=(lowerbounds[i],upperbounds[i])
    bounds.append(bb)

X0paths=[]  
X0path=[]
ypaths=[]
ypath=[]
def callbackF(Xi,convergence=0.05):
    X0path.append(Xi)
    ypath.append(optimingfunction(Xi,coercivitymodel,magnetomodel,magneticsaturationmodel))
    return 
strategies=["best1bin"]

indas=[[2],[2],[2],[2],[2],[2]]
indbs=[[7,8,12],[7,8,12],[None],[7,8,12],[7,8,12],[None]]
indcs=[[None],[7,8,12],[None],[None],[7,8,12],[None]]
indds=[[6,13],[6,13],[6,13],[6],[6],[6]]

for consi in range(6):
    inda=indas[consi]
    indb=indbs[consi]
    indc=indcs[consi]
    indd=indds[consi]

    for jj in inda :
        for ii in indb:
            for iii in indc:
                if ii==iii:
                    continue
                bb=bounds[:]
                bb[jj]=(0,0)
                if ii!=None:
                    bb[ii]=(0,0)
                if iii!=None:
                    bb[iii]=(0,0)
                for sind in indd:
                    bb[sind]=(0,0)

                for chosestrategy in strategies:
                    X0path=[]
                    ypath=[]
                    result=differential_evolution(optimingfunction,bb,args=(coercivitymodel,magnetomodel,magneticsaturationmodel), \
                                                  strategy=chosestrategy,popsize=30,mutation=(0.7,1.5),recombination=0.5, callback=callbackF, disp=1)
                    X0paths.append(X0path[:])
                    ypaths.append(ypath[:])
                    results.append(result)
                    datass.append(result['x'])

In [ ]:
'''get prediction value after optimization '''
predslist=[]
magstriction=[]
curietemppreds=[]
magneticsaturationpreds=[]
for data in datass:
    predss=coercivitymodel.predict(data.reshape(1,-1))
    magpred=magnetomodel.predict(data.reshape(1,-1))
    
    predslist.append(predss[0])
    magstriction.append(magpred[0])
    curietemppreds.append( curietempmodel.predict(data.reshape(1,-1))[0])
    magneticsaturationpreds.append( magneticsaturationmodel.predict(data.reshape(1,-1))[0])
magstriction=np.array(magstriction)
curietemppreds=np.array(curietemppreds)
magneticsaturationpreds=np.array(magneticsaturationpreds)
'''Sorting the optimized prediction ascending '''    
indexpred=np.argsort(predslist)
datass=np.array(datass)
datass2=datass[indexpred]
columnsname=coercivity.columns.values
optimizedframe=pd.DataFrame(data=datass2,columns=columnsname)
optimizedframe['coercivity']=sorted(predslist)
optimizedframe['magnetostriction']=magstriction[indexpred]
optimizedframe['curietemp']=curietemppreds[indexpred]
optimizedframe['magnetic-saturation']=magneticsaturationpreds[indexpred]
writer = pd.ExcelWriter('output_3.xlsx')
optimizedframe.to_excel(writer,'Sheet1')
writer.save()


## Fourth optimization strategy
#### Constrain magnetostriction < 3 ;    Minimizing (-magneticsatpred*4+coercivitypred)

In [ ]:
def optimingfunction(X,coercivitymodel,magnetomodel,magneticsaturationmodel):
    magnetopred=abs(magnetomodel.predict(X.reshape(1,-1), return_std=False))
    compsum=np.dot(A,X.reshape(-1,1))
    coercivitypred=coercivitymodel.predict(X.reshape(1,-1), return_std=False)
    magneticsatpred=magneticsaturationmodel.predict(X.reshape(1,-1), return_std=False)
    
    if compsum>100:
        return (compsum-100)**2+100
    if  magnetopred > 3 :
        return abs(magnetopred-3)**2+50-magneticsatpred
    
    return -magneticsatpred*4+coercivitypred

In [ ]:
'''Run optimizations based on different combination of element composition constraints'''


diffbet=[]

results=[]
datass=[]
from scipy.optimize import differential_evolution

bounds=[]
for i in range(lowerbounds.shape[0]):
    bb=(lowerbounds[i],upperbounds[i])
    bounds.append(bb)

X0paths=[]  
X0path=[]
ypaths=[]
ypath=[]
def callbackF(Xi,convergence=0.05):
    X0path.append(Xi)
    ypath.append(optimingfunction(Xi,coercivitymodel,magnetomodel,magneticsaturationmodel))
    return 
strategies=["best1bin"]

indas=[[2],[2],[2],[2],[2],[2]]
indbs=[[7,8,12],[7,8,12],[None],[7,8,12],[7,8,12],[None]]
indcs=[[None],[7,8,12],[None],[None],[7,8,12],[None]]
indds=[[6,13],[6,13],[6,13],[6],[6],[6]]

for consi in range(6):
    inda=indas[consi]
    indb=indbs[consi]
    indc=indcs[consi]
    indd=indds[consi]

    for jj in inda :
        for ii in indb:
            for iii in indc:
                if ii==iii:
                    continue
                bb=bounds[:]
                bb[jj]=(0,0)
                if ii!=None:
                    bb[ii]=(0,0)
                if iii!=None:
                    bb[iii]=(0,0)
                for sind in indd:
                    bb[sind]=(0,0)

                for chosestrategy in strategies:
                    X0path=[]
                    ypath=[]
                    result=differential_evolution(optimingfunction,bb,args=(coercivitymodel,magnetomodel,magneticsaturationmodel), \
                                                  strategy=chosestrategy,popsize=30,mutation=(0.7,1.5),recombination=0.5, callback=callbackF, disp=1)
                    X0paths.append(X0path[:])
                    ypaths.append(ypath[:])
                    results.append(result)
                    datass.append(result['x'])

In [ ]:
'''get prediction value after optimization '''
predslist=[]
magstriction=[]
curietemppreds=[]
magneticsaturationpreds=[]
for data in datass:
    predss=coercivitymodel.predict(data.reshape(1,-1))
    magpred=magnetomodel.predict(data.reshape(1,-1))
    
    predslist.append(predss[0])
    magstriction.append(magpred[0])
    curietemppreds.append( curietempmodel.predict(data.reshape(1,-1))[0])
    magneticsaturationpreds.append( magneticsaturationmodel.predict(data.reshape(1,-1))[0])
magstriction=np.array(magstriction)
curietemppreds=np.array(curietemppreds)
magneticsaturationpreds=np.array(magneticsaturationpreds)
'''Sorting the optimized prediction ascending '''    
indexpred=np.argsort(predslist)
datass=np.array(datass)
datass2=datass[indexpred]
columnsname=coercivity.columns.values
optimizedframe=pd.DataFrame(data=datass2,columns=columnsname)
optimizedframe['coercivity']=sorted(predslist)
optimizedframe['magnetostriction']=magstriction[indexpred]
optimizedframe['curietemp']=curietemppreds[indexpred]
optimizedframe['magnetic-saturation']=magneticsaturationpreds[indexpred]
writer = pd.ExcelWriter('output_4.xlsx')
optimizedframe.to_excel(writer,'Sheet1')
writer.save()


### Combine outputs together

In [ ]:
import glob
import pandas as pd

files = glob.glob('./output_*.xlsx', recursive=True)
frame=pd.DataFrame()
for file in files:
    frame=pd.concat([frame, pd.read_excel(file)])
frame=frame.reset_index(drop=True)

frame.to_excel('output_comb_together.xlsx')